In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data  = wine [['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42)

In [9]:

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 42)
print(train_input.shape, test_input.shape)
print('sub:', sub_input.shape, val_input.shape)

(5197, 3) (1300, 3)
sub: (4157, 3) (1040, 3)


In [10]:
# validation set로 모델 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
dt.score(sub_input, sub_target)

0.9971133028626413

In [13]:
# 사이킷런에서 교차검증은 따로 검증 세트를 분리하지 않아도 된다. 
# 기본적으로 5겹 검증이다
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00922799, 0.011271  , 0.00977349, 0.00810909, 0.0088141 ]), 'score_time': array([0.00202942, 0.0021174 , 0.00199795, 0.00120831, 0.00208235]), 'test_score': array([0.87019231, 0.84519231, 0.87006737, 0.85755534, 0.84215592])}


In [16]:
# 교차 검증으로 검증 점수의 최상을 가늠할 수 있다. 
import numpy as np
print(np.mean(scores['test_score']))

0.8570326497371733


In [18]:
# 훈련 세트 섞기 _ splitter
# 분류모델용 splitter _ StratifiedKFold
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8631940121535496


In [19]:
# 최적의 값 찾기_ GridSearchCV
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(), params, n_jobs = -1)

최적 값을 찾을 파라미터와 탐색할 값을 넘겨주었다.
gs 객체를 fit하면 결정 트리 모델의 min_impurity~ 의 값을 바꿔가며 5번 학습한다.
cv 는 기본적으로 5 이므로 5*5 = 25, 총 25번 학습한다. 
n_jobs 는 cpu 개수인데 -1 로 지정하면 전력을 다하는 거다. 

In [20]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [22]:
# 점수가 가장 높은 모델의 매개변수 조합이 best_estimator_ 속성에 저장되어 있다. 
dt=gs.best_estimator_
print(dt.score(train_input, train_target))

0.9613238406773138


In [24]:
# 최적의 매개변수 값을 확인해보자
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [27]:
# 교차 검증의 평균 점수 확인
print(gs.cv_results_['mean_test_score'])

[0.86800048 0.86549808 0.86530725 0.86780891 0.86761605]


In [29]:
# np.argmax() 와 교차 검증 mean 으로 파라미터 최적의 값 찾기
# np.argmax() 는 가장 큰 인덱스 값을 추출한다. 
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [32]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split':range(2,100,10)}
# range 는 끝값을 포함하고 arange는 끝값을 포함하지 않는다.

In [34]:
gs = GridSearchCV(DecisionTreeClassifier(),params,n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [35]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


In [37]:
# 교차검증 최대값
print(np.max(gs.cv_results_['mean_test_score']))

0.8687695269119715


In [43]:
# random search _ 난수 발생기
# 균등 분포에서 샘플링
# uniform : 실숫값, randint : 정수
from scipy.stats import uniform, randint

In [42]:
# 샘플링의 각 개수도 셀 수 있다. 
rgen = randint(0,10)
rgen.rvs(10)

array([8, 1, 9, 5, 5, 5, 8, 1, 2, 1], dtype=int64)

In [40]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 96, 102, 108, 115,  98,  92, 109,  75,  88, 117], dtype=int64))

In [41]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.22931065, 0.0759668 , 0.42705368, 0.48730856, 0.14790774,
       0.13539926, 0.75411171, 0.6964878 , 0.44152926, 0.07554654])

In [44]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split':randint(2,25),
         'min_samples_leaf': randint(1,25)}

In [45]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(), params,n_iter = 100, n_jobs = -1)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C85ABB51F0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C857BFC8F0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C85ABB7EC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C85ABB4E60>})

In [46]:
print(gs.best_params_)

{'max_depth': 46, 'min_impurity_decrease': 0.0004237154841317914, 'min_samples_leaf': 3, 'min_samples_split': 5}


In [49]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8689636854964092


In [52]:
# 최적의 모델 score 
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
